# Question 4: How long have they lived in their current residences? Are they homeowners or renters?

This notebook analyzes residential tenure for elderly residents in Allston-Brighton:

1. **Homeowner vs Renter Status**: Using `owner_occ` field from buildings table
2. **Length of Residency**: Minimum 5-year residency inferred from matching 2020 voter data to 2025 property data
3. **Geographic Distribution**: Breakdown by ward and precinct
4. **Demographic Analysis**: Breakdown by age group and income category

## Data Overview
- Total elderly: 7,396
- Mapped to buildings: 5,391
- Can determine homeowner/renter status for mapped elderly
- Can infer minimum 5-year residency for those matched to buildings


In [ ]:
import sys
import os

current_dir = os.getcwd()
if os.path.basename(current_dir) == 'notebooks':
    project_dir = os.path.dirname(current_dir)
else:
    parts = current_dir.split(os.sep)
    if 'fa25-team-a' in parts:
        idx = parts.index('fa25-team-a')
        project_dir = os.sep.join(parts[:idx+1])
    else:
        project_dir = os.path.dirname(os.path.dirname(current_dir))
web_app_path = os.path.join(project_dir, 'web_app')
sys.path.append(web_app_path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from config.database import get_db_connection, execute_query

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


## 1. Overall Elderly Population Statistics


In [2]:
query = """
SELECT 
    COUNT(DISTINCT v.res_id) as total_elderly,
    COUNT(DISTINCT CASE WHEN vbm.res_id IS NOT NULL THEN v.res_id END) as mapped_to_buildings,
    COUNT(DISTINCT CASE WHEN vbm.res_id IS NOT NULL AND b.owner_occ IS NOT NULL THEN v.res_id END) as with_tenure_data,
    AVG(v.age) as avg_age,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY v.age) as median_age
FROM voters v
LEFT JOIN voters_buildings_map vbm ON v.res_id = vbm.res_id
LEFT JOIN buildings b ON vbm.struct_id = b.struct_id
WHERE v.is_elderly = true
"""

df_overall = pd.DataFrame(execute_query(query, fetch_all=True))
print("Overall Elderly Population:")
print(df_overall.to_string(index=False))


Overall Elderly Population:
 total_elderly  mapped_to_buildings  with_tenure_data             avg_age  median_age
          7396                 5391              4390 75.1146804835924007        75.0


## 2. Homeowner vs Renter Analysis


In [9]:
query = """
SELECT DISTINCT ON (v.res_id)
    v.res_id,
    v.age,
    v.ward_id,
    v.precinct_id,
    b.owner_occ,
    CASE 
        WHEN b.owner_occ = 'Y' THEN 'Homeowner'
        WHEN b.owner_occ IS NULL THEN 'Unknown'
        ELSE 'Renter'
    END as tenure_status,
    b.total_value as property_value,
    CASE 
        WHEN b.total_value < 500000 THEN 'Low Value (<$500k)'
        WHEN b.total_value < 1000000 THEN 'Moderate Value ($500k-$1M)'
        WHEN b.total_value >= 1000000 THEN 'High Value ($1M+)'
        ELSE 'Unknown'
    END as value_category
FROM voters v
INNER JOIN voters_buildings_map vbm ON v.res_id = vbm.res_id
LEFT JOIN buildings b ON vbm.struct_id = b.struct_id
WHERE v.is_elderly = true
ORDER BY v.res_id
"""

df_tenure = pd.DataFrame(execute_query(query, fetch_all=True))

print("Homeowner vs Renter Analysis:")
print(f"Total elderly analyzed: {len(df_tenure):,}")
print(f"\nTenure Status Distribution:")
print(df_tenure['tenure_status'].value_counts())
print(f"\nTenure Status Percentages:")
print(df_tenure['tenure_status'].value_counts(normalize=True) * 100)

if df_tenure['property_value'].notna().any():
    print(f"\nProperty Value Statistics:")
    print(df_tenure[df_tenure['property_value'].notna()]['property_value'].describe())
    print(f"\nProperty Value by Tenure Status:")
    print(df_tenure.groupby('tenure_status')['property_value'].describe())


Homeowner vs Renter Analysis:
Total elderly analyzed: 5,391

Tenure Status Distribution:
tenure_status
Renter       2492
Homeowner    1898
Unknown      1001
Name: count, dtype: int64

Tenure Status Percentages:
tenure_status
Renter       46.225190
Homeowner    35.206826
Unknown      18.567984
Name: proportion, dtype: float64

Property Value Statistics:
count     3503
unique    1507
top       0.00
freq       529
Name: property_value, dtype: object

Property Value by Tenure Status:
              count unique         top freq
tenure_status                              
Homeowner      1840   1101  1403000.00    5
Renter         1663    454        0.00  529
Unknown           0      0         NaN  NaN


## 3. Length of Residency Analysis


In [10]:
query = """
SELECT DISTINCT ON (v.res_id)
    v.res_id,
    v.age,
    v.ward_id,
    v.precinct_id,
    CASE 
        WHEN EXISTS (SELECT 1 FROM voters_buildings_map vbm WHERE vbm.res_id = v.res_id) THEN true
        ELSE false
    END as has_building_match,
    CASE 
        WHEN EXISTS (SELECT 1 FROM voters_buildings_map vbm WHERE vbm.res_id = v.res_id) THEN '5+ Years (Matched)'
        ELSE 'Unknown'
    END as residency_stability,
    CASE 
        WHEN EXISTS (SELECT 1 FROM voters_buildings_map vbm WHERE vbm.res_id = v.res_id) THEN '5+ Years'
        ELSE 'Unknown'
    END as residency_duration
FROM voters v
WHERE v.is_elderly = true
ORDER BY v.res_id
"""

df_residency = pd.DataFrame(execute_query(query, fetch_all=True))

print("Length of Residency Analysis:")
print(f"Total elderly analyzed: {len(df_residency):,}")
print(f"\nElderly with Building Match (5+ years residency): {df_residency['has_building_match'].sum():,} ({df_residency['has_building_match'].sum()/len(df_residency)*100:.1f}%)")
print(f"\nResidency Duration Distribution:")
print(df_residency['residency_duration'].value_counts())
print(f"\nNote: Building match indicates minimum 5-year residency (2020 voter data → 2025 property data)")


Length of Residency Analysis:
Total elderly analyzed: 7,396

Elderly with Building Match (5+ years residency): 5,391 (72.9%)

Residency Duration Distribution:
residency_duration
5+ Years    5391
Unknown     2005
Name: count, dtype: int64

Note: Building match indicates minimum 5-year residency (2020 voter data → 2025 property data)


## 4. Combined Tenure and Residency Analysis


In [11]:
df_combined = df_tenure.merge(
    df_residency[['res_id', 'has_building_match', 'residency_duration']],
    on='res_id',
    how='left'
)

print("Combined Tenure and Residency Analysis:")
print(f"Total elderly with complete data: {len(df_combined):,}")
print(f"\nTenure Status by Residency Duration:")
print(pd.crosstab(df_combined['tenure_status'], df_combined['residency_duration'], margins=True))
print(f"\nResidency Duration by Tenure Status:")
print(pd.crosstab(df_combined['residency_duration'], df_combined['tenure_status'], margins=True))
print(f"\nKey Statistics:")
print(f"  - Homeowners with 5+ years residency: {((df_combined['tenure_status'] == 'Homeowner') & (df_combined['residency_duration'] == '5+ Years')).sum():,}")
print(f"  - Renters with 5+ years residency: {((df_combined['tenure_status'] == 'Renter') & (df_combined['residency_duration'] == '5+ Years')).sum():,}")
print(f"  - Homeowners with unknown residency: {((df_combined['tenure_status'] == 'Homeowner') & (df_combined['residency_duration'] == 'Unknown')).sum():,}")
print(f"  - Renters with unknown residency: {((df_combined['tenure_status'] == 'Renter') & (df_combined['residency_duration'] == 'Unknown')).sum():,}")


Combined Tenure and Residency Analysis:
Total elderly with complete data: 5,391

Tenure Status by Residency Duration:
residency_duration  5+ Years   All
tenure_status                     
Homeowner               1898  1898
Renter                  2492  2492
Unknown                 1001  1001
All                     5391  5391

Residency Duration by Tenure Status:
tenure_status       Homeowner  Renter  Unknown   All
residency_duration                                  
5+ Years                 1898    2492     1001  5391
All                      1898    2492     1001  5391

Key Statistics:
  - Homeowners with 5+ years residency: 1,898
  - Renters with 5+ years residency: 2,492
  - Homeowners with unknown residency: 0
  - Renters with unknown residency: 0


## 5. Tenure Analysis by Geographic Area


In [12]:
df_ward = df_combined.groupby('ward_id').agg({
    'res_id': 'count',
    'tenure_status': lambda x: x.value_counts().to_dict()
}).round(2)

df_ward_tenure = df_combined.groupby(['ward_id', 'tenure_status']).size().unstack(fill_value=0)
df_ward_tenure['total'] = df_ward_tenure.sum(axis=1)
df_ward_tenure['homeowner_pct'] = (df_ward_tenure.get('Homeowner', 0) / df_ward_tenure['total'] * 100).round(1)
df_ward_tenure['renter_pct'] = (df_ward_tenure.get('Renter', 0) / df_ward_tenure['total'] * 100).round(1)

print("Tenure Status by Ward:")
print(df_ward_tenure.to_string())


Tenure Status by Ward:
tenure_status  Homeowner  Renter  Unknown  total  homeowner_pct  renter_pct
ward_id                                                                    
21                   395    1820      879   3094           12.8        58.8
22                  1503     672      122   2297           65.4        29.3


In [13]:
df_precinct_tenure = df_combined.groupby(['precinct_id', 'tenure_status']).size().unstack(fill_value=0)
df_precinct_tenure['total'] = df_precinct_tenure.sum(axis=1)
df_precinct_tenure = df_precinct_tenure.sort_values('total', ascending=False).head(15)

print("Top 15 Precincts by Elderly Count - Tenure Status:")
print(df_precinct_tenure.to_string())


Top 15 Precincts by Elderly Count - Tenure Status:
tenure_status  Homeowner  Renter  Unknown  total
precinct_id                                     
13                   164     628      175    967
12                   156     297      190    643
10                   149     352       28    529
7                    131      74      260    465
9                    121     195       98    414
11                   220     104        4    328
1                    160     140        3    303
8                    110     138       27    275
3                    132      84       53    269
4                    135      78       25    238
2                    138      75        4    217
6                    139      70        3    212
5                     61      91       35    187
16                    68      43       71    182
15                     4      82       12     98


## 6. Tenure Analysis by Age Group


In [14]:
df_combined['age_group'] = pd.cut(
    df_combined['age'],
    bins=[0, 69, 79, 89, 150],
    labels=['62-69', '70-79', '80-89', '90+']
)

df_age_tenure = df_combined.groupby(['age_group', 'tenure_status']).size().unstack(fill_value=0)
df_age_tenure['total'] = df_age_tenure.sum(axis=1)
df_age_tenure['homeowner_pct'] = (df_age_tenure.get('Homeowner', 0) / df_age_tenure['total'] * 100).round(1)
df_age_tenure['renter_pct'] = (df_age_tenure.get('Renter', 0) / df_age_tenure['total'] * 100).round(1)

print("Tenure Status by Age Group:")
print(df_age_tenure.to_string())


Tenure Status by Age Group:
tenure_status  Homeowner  Renter  Unknown  total  homeowner_pct  renter_pct
age_group                                                                  
62-69                722     712      404   1838           39.3        38.7
70-79                725     833      405   1963           36.9        42.4
80-89                332     638      152   1122           29.6        56.9
90+                  119     309       40    468           25.4        66.0


/var/folders/37/0zy_nm5d0rv5b6vp7z9r6hfh0000gp/T/ipykernel_53108/2805707320.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_age_tenure = df_combined.groupby(['age_group', 'tenure_status']).size().unstack(fill_value=0)


## 7. Tenure Analysis by Income Category


In [15]:
query = """
SELECT DISTINCT ON (v.res_id)
    v.res_id,
    COALESCE(ct.median_income, 0) as median_income,
    CASE 
        WHEN COALESCE(ct.median_income, 0) < 50000 THEN 'Low Income (<$50k)'
        WHEN COALESCE(ct.median_income, 0) < 75000 THEN 'Moderate Income ($50k-$75k)'
        ELSE 'Higher Income ($75k+)'
    END as income_category
FROM voters v
INNER JOIN voters_buildings_map vbm ON v.res_id = vbm.res_id
LEFT JOIN census_tracts ct ON ST_Within(
    ST_SetSRID(ST_MakePoint(v.longitude, v.latitude), 4326),
    ct.geometry
)
WHERE v.is_elderly = true
    AND v.latitude IS NOT NULL
    AND v.longitude IS NOT NULL
ORDER BY v.res_id, ct.median_income
"""

df_income = pd.DataFrame(execute_query(query, fetch_all=True))

df_combined_income = df_combined.merge(
    df_income[['res_id', 'income_category']],
    on='res_id',
    how='left'
)

df_income_tenure = df_combined_income.groupby(['income_category', 'tenure_status']).size().unstack(fill_value=0)
df_income_tenure['total'] = df_income_tenure.sum(axis=1)
df_income_tenure['homeowner_pct'] = (df_income_tenure.get('Homeowner', 0) / df_income_tenure['total'] * 100).round(1)
df_income_tenure['renter_pct'] = (df_income_tenure.get('Renter', 0) / df_income_tenure['total'] * 100).round(1)

print("Tenure Status by Income Category:")
print(df_income_tenure.to_string())


Tenure Status by Income Category:
tenure_status                Homeowner  Renter  Unknown  total  homeowner_pct  renter_pct
income_category                                                                          
Higher Income ($75k+)             1415    1916      666   3997           35.4        47.9
Low Income (<$50k)                 324     341      251    916           35.4        37.2
Moderate Income ($50k-$75k)        159     235       81    475           33.5        49.5


## 8. Comprehensive Tenure Profile


In [16]:
df_comprehensive = df_combined.merge(
    df_income[['res_id', 'income_category', 'median_income']],
    on='res_id',
    how='left'
)

print("Comprehensive Tenure Profile:")
print(f"Total elderly with complete profile: {len(df_comprehensive):,}")
print(f"\nSummary Statistics:")
print(f"  - Homeowners: {(df_comprehensive['tenure_status'] == 'Homeowner').sum():,} ({(df_comprehensive['tenure_status'] == 'Homeowner').sum()/len(df_comprehensive)*100:.1f}%)")
print(f"  - Renters: {(df_comprehensive['tenure_status'] == 'Renter').sum():,} ({(df_comprehensive['tenure_status'] == 'Renter').sum()/len(df_comprehensive)*100:.1f}%)")
print(f"  - With 5+ years residency: {df_comprehensive['has_building_match'].sum():,} ({df_comprehensive['has_building_match'].sum()/len(df_comprehensive)*100:.1f}%)")
print(f"  - Homeowners with 5+ years: {((df_comprehensive['tenure_status'] == 'Homeowner') & df_comprehensive['has_building_match']).sum():,}")
print(f"  - Renters with 5+ years: {((df_comprehensive['tenure_status'] == 'Renter') & df_comprehensive['has_building_match']).sum():,}")


Comprehensive Tenure Profile:
Total elderly with complete profile: 5,391

Summary Statistics:
  - Homeowners: 1,898 (35.2%)
  - Renters: 2,492 (46.2%)
  - With 5+ years residency: 5,391 (100.0%)
  - Homeowners with 5+ years: 1,898
  - Renters with 5+ years: 2,492


## 9. Export Results


In [ ]:
output_dir = os.path.join(project_dir, 'data', 'processed', 'elderly_analysis')
os.makedirs(output_dir, exist_ok=True)

df_tenure.to_csv(f'{output_dir}/tenure_homeowner_renter.csv', index=False)
df_residency.to_csv(f'{output_dir}/tenure_residency_length.csv', index=False)
df_combined.to_csv(f'{output_dir}/tenure_combined.csv', index=False)
df_comprehensive.to_csv(f'{output_dir}/tenure_comprehensive.csv', index=False)
df_ward_tenure.to_csv(f'{output_dir}/tenure_by_ward.csv', index=True)
df_precinct_tenure.to_csv(f'{output_dir}/tenure_by_precinct.csv', index=True)
df_age_tenure.to_csv(f'{output_dir}/tenure_by_age_group.csv', index=True)
df_income_tenure.to_csv(f'{output_dir}/tenure_by_income.csv', index=True)

print("✅ All tenure analysis files exported to:", output_dir)


✅ All tenure analysis files exported to: /Users/Studies/Projects/ds-abcdc-allston/fa25-team-a/data/processed/elderly_analysis
